In [ ]:
import numpy as np
import pandas as pd
import numpy_financial as npf
import datetime


In [ ]:
Interest_Rate = 0.04
Years = 30
Payments_Year = 12 # monthly payments
Principal = 200000
start_date = datetime.datetime(2016, 1, 1)

In [ ]:
# Payment against loan principal plus interest

# As the payment is monthly the interest rate is divided and the years is multiply. The payment is done at the end of the month.

# when="end" is default
monthly_payment = npf.pmt(Interest_Rate/Payments_Year, Years*Payments_Year, Principal, when="end")
print(monthly_payment)

In [ ]:
total_payment = monthly_payment * 12 * Years
total_payment

In [ ]:
# Interest portion of a payment
per = 1
interest_payment = npf.ipmt(Interest_Rate/Payments_Year, per, Years*Payments_Year, Principal)

interest_payment

In [ ]:
principal_payment = npf.ppmt(Interest_Rate/Payments_Year, per, Years*Payments_Year, Principal)

principal_payment